In [1]:
import os
import numpy as np
import trimesh
from scipy.ndimage import zoom
from tqdm import tqdm
!pip install kagglehub
import kagglehub

In [2]:
# Download latest version
path = kagglehub.dataset_download("balraj98/modelnet40-princeton-3d-object-dataset")

print("Path to dataset files:", path)

100%|██████████| 1.91G/1.91G [1:27:52<00:00, 388kB/s]

Extracting files...


Path to dataset files: C:\Users\amirh\.cache\kagglehub\datasets\balraj98\modelnet40-princeton-3d-object-dataset\versions\1


In [7]:
dataset_root = os.path.join(path, "ModelNet40")  # ⚠️ مطمئن شو path تعریف شده
selected_classes = [
    'airplane', 'bathtub', 'bed', 'bench', 'bookshelf', 'bottle', 'bowl', 'car',
    'chair', 'cone', 'cup', 'curtain', 'desk', 'door', 'dresser', 'flower_pot',
    'glass_box', 'guitar', 'keyboard', 'lamp', 'laptop', 'mantel', 'monitor',
    'night_stand', 'person', 'piano', 'plant', 'radio', 'range_hood', 'sink',
    'sofa', 'stairs', 'stool', 'table', 'tent', 'toilet', 'tv_stand', 'vase',
    'wardrobe', 'xbox'
]
voxel_dim = 32  # اندازه شبکه وکسل

In [15]:
import trimesh
import numpy as np

def mesh_to_pointcloud(path_to_off_file, num_points=1024):
    mesh = trimesh.load(path_to_off_file)
    if not isinstance(mesh, trimesh.Trimesh):
        mesh = mesh.dump().sum()
    
    # Sample points uniformly from surface
    points, _ = trimesh.sample.sample_surface(mesh, num_points)
    
    return points.astype(np.float32)  # shape: (num_points, 3)

In [17]:
def load_pointcloud_dataset(dataset_path, num_points=1024, selected_classes=None, split='train'):
    X, y = [], []

    if selected_classes is None:
        selected_classes = sorted(os.listdir(dataset_path))
    
    label_map = {cls_name: idx for idx, cls_name in enumerate(selected_classes)}

    for cls_name in tqdm(selected_classes, desc="Classes"):
        class_path = os.path.join(dataset_path, cls_name, split)
        if not os.path.exists(class_path):
            continue
        
        file_list = [f for f in os.listdir(class_path) if f.endswith('.off')]
        for file_name in tqdm(file_list, desc=f"{cls_name} ({split})", leave=False):
            file_path = os.path.join(class_path, file_name)
            try:
                pc = mesh_to_pointcloud(file_path, num_points)
                X.append(pc)
                y.append(label_map[cls_name])
            except Exception as e:
                print(f"⚠️ Error in {file_path}: {e}")
    
    X = np.array(X, dtype=np.float32)
    y = np.array(y, dtype=np.int32)
    return X, y, label_map

In [19]:
num_points = 1024  # یا 2048، بسته به منابع

X_train_pc, y_train_pc, label_map = load_pointcloud_dataset(
    dataset_root, num_points=num_points, selected_classes=selected_classes, split='train'
)

X_test_pc, y_test_pc, _ = load_pointcloud_dataset(
    dataset_root, num_points=num_points, selected_classes=selected_classes, split='test'
)

curtain (train):  47%|████▋     | 65/138 [00:09<00:29,  2.44it/s]C:\Users\amirh\anaconda3\Lib\site-packages\trimesh\grouping.py:99: RuntimeWarning: invalid value encountered in cast
  stacked = np.column_stack(stacked).round().astype(np.int64)

Classes: 100%|██████████| 40/40 [05:25<00:00,  8.15s/it]    


In [21]:
np.save("MN40_X_train_pc.npy", X_train_pc)
np.save("MN40_y_train_pc.npy", y_train_pc)
np.save("MN40_X_test_pc.npy", X_test_pc)
np.save("MN40_y_test_pc.npy", y_test_pc)